# Preparing model architecture

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [3]:
class AttentionLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="attention_weights",
                                 shape=(input_shape[-1], 1),
                                 initializer="glorot_uniform",
                                 trainable=True)
        super().build(input_shape)

    def call(self, inputs):
        # Compute attention scores
        e = tf.matmul(inputs, self.W)
        alpha = tf.nn.softmax(e, axis=1)
        context = tf.reduce_sum(alpha * inputs, axis=1)
        return context, tf.squeeze(alpha, -1)  # output attention weights too

def build_attention_model(input_dim, num_classes):
    inputs = tf.keras.Input(shape=(input_dim,))
    
    # Expand dims to simulate time steps for attention (shape: batch, time, features)
    x = tf.expand_dims(inputs, axis=1)
    
    attention_output, attention_weights = AttentionLayer()(x)
    
    x = layers.Dense(64, activation='relu')(attention_output)
    x = layers.Dense(32, activation='relu')(x)
    output = layers.Dense(num_classes, activation='softmax')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=[output, attention_weights])
    return model

# Creating Entity method for model directory

In [8]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelPreparationConfig:
    root_dir: Path
    training_independent_data_path: Path
    training_dependent_data_path: Path
    model_saved_path: Path

# Preparing Configuration Manager

In [6]:
import os
os.chdir('../')

In [7]:
from src.No_More_Lapses.constants import *
from src.No_More_Lapses.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_preparation_config(self) -> ModelPreparationConfig:
        config = self.config.model_trainer

        create_directories([config.root_dir])
        
        data_transformation_config = ModelPreparationConfig(
            root_dir=config.root_dir,
            training_independent_data_path = config.training_independent_data_path,
            training_dependent_data_path = config.training_dependent_data_path,
            model_saved_path = config.model_saved_path
        )

        return data_transformation_config

### Testing the configuration manager

In [10]:
a = ConfigurationManager()

[2025-03-24 11:17:20,994: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-03-24 11:17:20,998: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-24 11:17:21,000: INFO: common: created directory at: artifacts]


In [11]:
a.get_model_preparation_config()

[2025-03-24 11:17:28,097: INFO: common: created directory at: artifacts/model_trainer]


ModelPreparationConfig(root_dir='artifacts/model_trainer', training_independent_data_path='artifacts/transformed_data/trainining_data/X_train.csv', training_dependent_data_path='artifacts/transformed_data/trainining_data/y_train.csv', model_saved_path='artifacts/model_trainer/attention_model.h5')

# Model Preparation